In [20]:
# 基于用户的协同过滤推荐算法
import os
import random
import math
from operator import itemgetter
import shelve
from contextlib import closing
import pandas as pd
import numpy as np
from datetime import datetime
import glob
import pandas as pd 

In [2]:
# 原始方法 read file 
def load_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i == 0:
                # 去掉文件第一行的title
                continue
            yield line.strip('\r\n')
    print('Load %s success!' % filename)

def get_dataset(filename, pivot=0.75):
    # train data set
    trainSet = {}
    trainSet_len = 0

    # test data set 
    testSet = {}
    testSet_len = 0

    # 加载文件， 按行读取
    for line in load_file(filename):
        # 读取列属性
        user, movie, rating, timestamp = line.split(',')
        # string transform number(int or float)
        user = int(user)
        movie = int(movie)
        rating = float(rating)

        # 数据划分测试集合和数据集合 (0,1) < (0,pivot)
        if random.random() < pivot:
            trainSet.setdefault(user, {})
            trainSet[user][movie] = rating
            trainSet_len += 1
        else:
            testSet.setdefault(user, {})
            testSet[user][movie] = rating
            testSet_len += 1
            
    print('Split trainingSet and testSet success!')
    print('user - movie - matrix')
    print('TrainSet = %s' % trainSet_len)
    print('TestSet = %s' % testSet_len)
    return trainSet, testSet


In [3]:
path = r"ratings.csv"
trainSet, testSet = get_dataset(path)

Load ratings.csv success!
Split trainingSet and testSet success!
user - movie - matrix
TrainSet = 75685
TestSet = 25151


数据处理 

user movie 


In [ ]:
def calc_user_sim(self):
        # 构建“电影-用户”倒排索引
        # key = movieID, value = list of userIDs who have seen this movie
        print('Building movie-user table ...')
        movie_user = {}
        for user, movies in self.trainSet.items():
            for movie in movies:
                if movie not in movie_user:
                    movie_user[movie] = set()
                movie_user[movie].add(user)
        print('Build movie-user table success!')

        self.movie_count = len(movie_user)
        print('Total movie number = %d' % self.movie_count)

        print('Build user co-rated movies matrix ...')
        for movie, users in movie_user.items():
            for u in users:
                for v in users:
                    if u == v:
                        continue
                    self.user_sim_matrix.setdefault(u, {})
                    self.user_sim_matrix[u].setdefault(v, 0)
                    self.user_sim_matrix[u][v] += 1
        print('Build user co-rated movies matrix success!')

        # 计算相似性, 用户之间得相似性
        print('Calculating user similarity matrix ...')
        for u, related_users in self.user_sim_matrix.items():
            for v, count in related_users.items():
                self.user_sim_matrix[u][v] = count / math.sqrt(len(self.trainSet[u]) * len(self.trainSet[v]))
        print('Calculate user similarity matrix success!')

In [18]:
def cal_movie_user(trainSet):
    user_sim_matrix = {}
    user_movie = {}
    for user, movies in trainSet.items():
        for movie in movies:
            if user not in user_movie:
                user_movie.setdefault(user, set())
            user_movie[user].add(movie)
    print('Build user-movie table success!')
    
    users = user_movie.keys()

    for u in users:
        for v in users:
            user_sim_matrix.setdefault(u, {})
            user_sim_matrix[u].setdefault(v, 0)
            if u == v:
                user_sim_matrix[u][v] = 1
                continue
            u_set = set(trainSet.get(u))
            v_set = set(trainSet.get(v))
            top = set.intersection(u_set, v_set)
            bottom = set.union(u_set, v_set)
            user_sim_matrix[u][v] = len(top) / len(bottom)
    print('Build user-sim table success!')
    return user_sim_matrix, user_movie

user_sim, users = cal_movie_user(trainSet)

Build user-movie table success!
Build user-sim table success!


In [32]:
# trainSet.get(1)

In [30]:
con = set.intersection(set(trainSet.get(1)), set(trainSet.get(2)))

u,v = [],[]
for i in con:
    u.append(trainSet.get(1).get(i))
    v.append(trainSet.get(2).get(i))
print(u,v)

[5.0] [4.0]


In [22]:
pd.DataFrame(user_sim)

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,1.000000,0.005236,0.025773,0.080268,0.041237,0.054830,0.041985,0.060914,0.015306,0.003745,...,0.026201,0.067194,0.065139,0.025210,0.056075,0.043744,0.101754,0.112188,0.020619,0.031789
2,0.005236,1.000000,0.000000,0.005747,0.018868,0.007874,0.016260,0.016667,0.000000,0.025641,...,0.035714,0.008264,0.002729,0.000000,0.000000,0.002283,0.006061,0.006144,0.020408,0.012295
3,0.025773,0.000000,1.000000,0.005525,0.000000,0.003817,0.000000,0.000000,0.000000,0.000000,...,0.010753,0.007812,0.005420,0.000000,0.005076,0.003401,0.011696,0.006079,0.000000,0.003024
4,0.080268,0.005747,0.005525,1.000000,0.039326,0.037534,0.040650,0.032258,0.005525,0.016194,...,0.033175,0.054167,0.098985,0.022523,0.018987,0.055439,0.042105,0.056452,0.005556,0.021918
5,0.041237,0.018868,0.000000,0.039326,1.000000,0.076923,0.015038,0.183333,0.000000,0.000000,...,0.010417,0.157895,0.019152,0.081633,0.036082,0.016018,0.047619,0.023077,0.071429,0.007064
6,0.054830,0.007874,0.003817,0.037534,0.076923,1.000000,0.021212,0.114286,0.000000,0.006061,...,0.000000,0.184397,0.047566,0.157895,0.033333,0.034156,0.064789,0.086466,0.052209,0.014370
7,0.041985,0.016260,0.000000,0.040650,0.015038,0.021212,1.000000,0.028986,0.007634,0.046875,...,0.050000,0.035714,0.017456,0.017241,0.075099,0.046943,0.055556,0.082353,0.007692,0.038873
8,0.060914,0.016667,0.000000,0.032258,0.183333,0.114286,0.028986,1.000000,0.000000,0.007353,...,0.009709,0.241071,0.027322,0.130000,0.050505,0.017045,0.070175,0.048362,0.175439,0.011066
9,0.015306,0.000000,0.000000,0.005525,0.000000,0.000000,0.007634,0.000000,1.000000,0.007937,...,0.021739,0.007812,0.002703,0.009804,0.010204,0.009122,0.000000,0.012232,0.000000,0.008105
10,0.003745,0.025641,0.000000,0.016194,0.000000,0.006061,0.046875,0.007353,0.007937,1.000000,...,0.065359,0.005076,0.004957,0.005848,0.038911,0.021413,0.012552,0.023810,0.000000,0.026037


In [15]:
def cal_movie_user(trainSet):
    movie_user = {}
    user_sim_matrix = {}
    for user, movies in trainSet.items():
        for movie in movies:
            if movie not in movie_user:
                movie_user.setdefault(movie, set())
            movie_user[movie].add(user)
    print('Build movie-user table success!')
    
    for movie, users in movie_user.items:
        for u in users:
            for v in user:
                if u == v:
                    continue
                # 防止 user_sim_matrix [u][v] 不存在
                user_sim_matrix.setdefault(u,{})
                user_sim_matrix[u].setdefault(v, 0)
                user_sim_matrix[u][v] += 1
    # 得到用户-用户表，matrix[u][v]共同看过的电影数量

    return movie_user

def build_user_movie(trainSet):
    user_movie = {}
    for user, movies in trainSet.items():
        for movie in movies:
            if user not in user_movie:
                user_movie.setdefault(user, set())
            user_movie[user].add(movie)
    return user_movie

data = cal_movie_user(trainSet)
print(len(data))
for i in range(len(data)):
    try:
        print(i, data[i])
    except KeyError:
        continue
    if i > 3:
        break

8743
1 {514, 5, 522, 525, 15, 528, 529, 18, 19, 21, 533, 534, 27, 541, 31, 33, 550, 40, 44, 46, 559, 560, 561, 50, 54, 567, 57, 570, 573, 63, 64, 66, 579, 68, 71, 584, 587, 76, 78, 590, 82, 596, 86, 600, 89, 90, 601, 604, 93, 605, 606, 96, 607, 608, 609, 610, 103, 107, 119, 121, 124, 130, 132, 134, 135, 137, 141, 144, 145, 153, 155, 156, 160, 161, 166, 167, 169, 171, 177, 182, 186, 191, 201, 202, 206, 213, 214, 217, 219, 220, 223, 226, 229, 233, 234, 239, 240, 254, 263, 264, 266, 269, 270, 273, 275, 276, 277, 279, 280, 282, 283, 288, 291, 292, 298, 304, 307, 314, 322, 323, 328, 330, 332, 334, 336, 337, 339, 341, 347, 357, 359, 364, 367, 372, 373, 378, 380, 381, 382, 385, 389, 401, 411, 412, 414, 420, 432, 434, 436, 438, 443, 448, 453, 456, 462, 468, 474, 476, 477, 480, 483, 484, 488, 500, 509}
2 {512, 517, 6, 8, 523, 525, 18, 19, 20, 534, 27, 559, 561, 563, 570, 62, 68, 586, 590, 82, 594, 599, 600, 604, 93, 94, 605, 608, 103, 107, 117, 122, 125, 135, 140, 149, 153, 160, 169, 177, 186, 

In [21]:
def count_user(trainSet):
    # 统计电影被看的次数
    movie_popular = {}
    user_list = []
    for user, movies in trainSet.items():
        if user not in user_list:
            user_list.append(user)
        for movie in movies:
            if movie not in movie_popular:
                movie_popular[movie] = 0
            movie_popular[movie] += 1
    
    print("Total user number = %d" % len(user_list))
    print("Total movie number = %d" % len(movie_popular))
    return movie_popular, sorted(user_list)

movie_popular, user_list = count_user(trainSet)

Total user number = 610
Total movie number = 8784


In [35]:
np.array(user_list)
np.sort(np.array(list(movie_popular.keys())))

np.array

array([     1,      2,      3, ..., 193583, 193587, 193609])

In [34]:
a = {(1,2):2, (2,1):3, (1, 2):4}
a 
a.get(2,1)

1

In [ ]:
def sim_movie(trainSet, movie_popular):
    sim = tl.Similarity()
    movie_sim_matrix = {}
    i = 0
    max = len(movie_popular)
    # 遍历训练数据，获得用户对有过的行为的物品

    movie_list = list(movie_popular.keys())

    for i1 in range(0, len(movie_list)):
        # 遍历该用户每件物品项
        for i2 in range(i1, len(movie_list)):
            # 遍历该用户每件物品项
            
            m1, m2 = movie_list[i1], movie_list[i2]

            movie_sim_matrix.setdefault(m1, {})
            movie_sim_matrix.setdefault(m2, {})
            if m1 == m2:
                movie_sim_matrix[m1].setdefault(m2,1)
                continue 
                
            u1 = trainSet.get(m1)
            u2 = trainSet.get(m2)
            
            simi = sim.cosine_similarity(u1,u2)

            movie_sim_matrix[m1].setdefault(m2, simi)
            movie_sim_matrix[m2].setdefault(m1, simi)
        i += 1
        print("\r{0}%".format(round(i/max*100,4)), end='')
        
    print()
    # movie and movie 矩阵
    print("Build 同现矩阵co-rated users matrix success!")
    return movie_sim_matrix

In [ ]:
path = r"F:\毕业设计\ItemCF和UserCF协同过滤算法参考代码\MovieRecommendation-master\UserCF" 
glob.glob(path+'\*data*')
from contextlib import closing
import shelve

def data_init(path):
    #self.trainSet, self.testSet = load_data.get_dataset(path)
    #  读取测试数据 
    with closing(shelve.open(path,'r')) as shelf:
        trainSet = shelf['train']
        testSet = shelf['test']

    print("train data",len(trainSet),"- test data",len(testSet))
    return trainSet, testSet
trainSet, testSet = data_init(path+'\\time0data')


In [ ]:
a = pd.Series(trainSet['1'])
b = pd.Series(trainSet['2'])
print(type(a),type(b))


In [ ]:
pd.DataFrame(trainSet)

In [ ]:
def sim_by_series(a,b):
    mul_a_b = a.astype(np.float).mul(b.astype(np.float), fill_value=0).sum()
    a2 = np.sqrt(np.sum(a.astype(np.float).mul(a.astype(np.float),fill_value=0)))
    b2 = np.sqrt(np.sum(b.astype(np.float).mul(b.astype(np.float),fill_value=0)))
    return mul_a_b / (a2 * b2)

# 余弦相似度
def cosine(x,y):
    x = pd.Series(x)
    y = pd.Series(y) 
    xy = x.astype(float).mul(y.astype(float), fill_value=0)
    x = np.linalg.norm(x.values.astype(float),2)
    y = np.linalg.norm(y.values.astype(float),2)
    return xy.sum() / (x*y)


cosine(trainSet.get('3'),trainSet.get('2'))


In [ ]:
# 计算电影之间的相似度, 利用其他算法
def calc_movie_sim_test(trainSet, func):
    movie_sim_matrix = {}

    for user1 in trainSet.keys():
        movie_sim_matrix.setdefault(user1, {})
        for user2 in trainSet.keys():
            if user1 == user2:
                movie_sim_matrix.get(user1).setdefault(user2, 1)
            else:
                movie_sim_matrix.get(user1).setdefault(user2, func(trainSet.get(user1), trainSet.get(user2)))
    # movie{1,2}=sim
    return  movie_sim_matrix


In [ ]:
func = cosine
pd.DataFrame(calc_movie_sim_test(trainSet, func))

In [ ]:
sim_user = np.array(np.zeros((1000,1000)))

for i in trainSet.keys():
    a = pd.Series(trainSet[i])
    for j in trainSet.keys():
#         print(i,j)
        b = pd.Series(trainSet[j])
        sim_user[int(i),int(j)] = sim_by_series(a,b)
        pass
sim_user

In [ ]:
pd.DataFrame(sim_user)

In [ ]:
np.sum(mul_a_b) / (a2*b2)

In [ ]:
sim_user = np.array(np.zeros((10,10)))
print(sim_user)

In [ ]:
import sys,time
for i in range(100):
    sys.stdout.write(‘\r%s%%‘%(i+1))
    sys.stdout.flush()
    time.sleep(0.1)

In [33]:
# coding:utf-8

from threading import Thread  # 导入线程函数
from time import sleep  # 导入时间休眠函数


def task1(n):  # 定义任务1
    for x in range(n):  # 遍历数组n里的x
        print("正在运行任务1,打印：", x)  # 打印正则运行任务1，打印x的值
    sleep(0.5)  # 时间休眠0.5秒


def task2():  # 定义任务2
    list1 = ["北京欢迎您", "红日", "故宫的记忆", "义勇军进行曲"]  # 定义列表1位4首歌曲名
    for info in list1:  # 遍历list1里的信息
        print("正在运行任务2，听音乐：", info)  # 打印正在运行任务2，听音乐:音乐名称
    sleep(0.6)  # 时间休眠0.6秒


def main():  # 定义main函数
    t1 = Thread(target=task1, args=(6,))  # 定义线程t1，线程任务为调用task1函数，task1函数的参数是6
    t2 = Thread(target=task2)  # 定义线程t2，线程任务为调用task2函数，task2函数无参数
    t1.start()  # 开始运行t1线程
    t2.start()  # 开始运行t2线程



main()  # 调用main函数

正在运行任务1,打印： 0
正在运行任务1,打印： 1
正在运行任务1,打印： 2
正在运行任务1,打印： 3
正在运行任务1,打印： 4
正在运行任务1,打印： 5
正在运行任务2，听音乐： 北京欢迎您
正在运行任务2，听音乐： 红日
正在运行任务2，听音乐： 故宫的记忆
正在运行任务2，听音乐： 义勇军进行曲
